<a href="https://colab.research.google.com/github/m73abbasi/CS-SBU-DataMining-Msc-projects/blob/main/MohammadRezaAbbasi-98422101/project%203/mobile_price_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# mobile-price-classification

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split as tts
from sklearn import svm
from sklearn.metrics import classification_report, confusion_matrix

In [3]:
# read data
train = pd.read_csv('/content/drive/MyDrive/train.csv')
test = pd.read_csv('/content/drive/MyDrive/test.csv')
pd.set_option('display.max_columns',None)

In [4]:
train.head()

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,pc,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range
0,842,0,2.2,0,1,0,7,0.6,188,2,2,20,756,2549,9,7,19,0,0,1,1
1,1021,1,0.5,1,0,1,53,0.7,136,3,6,905,1988,2631,17,3,7,1,1,0,2
2,563,1,0.5,1,2,1,41,0.9,145,5,6,1263,1716,2603,11,2,9,1,1,0,2
3,615,1,2.5,0,0,0,10,0.8,131,6,9,1216,1786,2769,16,8,11,1,0,0,2
4,1821,1,1.2,0,13,1,44,0.6,141,2,14,1208,1212,1411,8,2,15,1,1,0,1


In [5]:
test.head()

,id,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,pc,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi
0,1,1043,1,1.8,1,14,0,5,0.1,193,3,16,226,1412,3476,12,7,2,0,1,0
1,2,841,1,0.5,1,4,1,61,0.8,191,5,12,746,857,3895,6,0,7,1,0,0
2,3,1807,1,2.8,0,1,0,27,0.9,186,3,4,1270,1366,2396,17,10,10,0,1,1
3,4,1546,0,0.5,1,18,1,25,0.5,96,8,20,295,1752,3893,10,0,7,1,1,0
4,5,1434,0,1.4,0,11,1,49,0.5,108,6,18,749,810,1773,15,8,7,1,0,1


In [6]:
# check number of train
train.shape

(2000, 21)

In [7]:
# check number of test
test.shape

(1000, 21)

In [8]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   battery_power  2000 non-null   int64  
 1   blue           2000 non-null   int64  
 2   clock_speed    2000 non-null   float64
 3   dual_sim       2000 non-null   int64  
 4   fc             2000 non-null   int64  
 5   four_g         2000 non-null   int64  
 6   int_memory     2000 non-null   int64  
 7   m_dep          2000 non-null   float64
 8   mobile_wt      2000 non-null   int64  
 9   n_cores        2000 non-null   int64  
 10  pc             2000 non-null   int64  
 11  px_height      2000 non-null   int64  
 12  px_width       2000 non-null   int64  
 13  ram            2000 non-null   int64  
 14  sc_h           2000 non-null   int64  
 15  sc_w           2000 non-null   int64  
 16  talk_time      2000 non-null   int64  
 17  three_g        2000 non-null   int64  
 18  touch_sc

In [9]:
# normalizing numeric data
for cols in train.columns:
  if train[cols].dtype == 'int64' or train[cols].dtype == 'float64':
    if cols != "price_range":
      train[cols] = ((train[cols]-train[cols].mean())/(train[cols].std()))

In [10]:
# delete outlier for train data
for cols in train.columns:
  if train[cols].dtype == 'int64' or train[cols].dtype == 'float64':
    upper_range = train[cols].mean() + 3 * train[cols].std()
    lower_range = train[cols].mean() - 3 * train[cols].std()
    idx = train[(train[cols] > upper_range) | (train[cols] < lower_range)].index
    train = train.drop(idx)
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1988 entries, 0 to 1999
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   battery_power  1988 non-null   float64
 1   blue           1988 non-null   float64
 2   clock_speed    1988 non-null   float64
 3   dual_sim       1988 non-null   float64
 4   fc             1988 non-null   float64
 5   four_g         1988 non-null   float64
 6   int_memory     1988 non-null   float64
 7   m_dep          1988 non-null   float64
 8   mobile_wt      1988 non-null   float64
 9   n_cores        1988 non-null   float64
 10  pc             1988 non-null   float64
 11  px_height      1988 non-null   float64
 12  px_width       1988 non-null   float64
 13  ram            1988 non-null   float64
 14  sc_h           1988 non-null   float64
 15  sc_w           1988 non-null   float64
 16  talk_time      1988 non-null   float64
 17  three_g        1988 non-null   float64
 18  touch_sc

In [11]:
# delete outlier for test data
for cols in test.columns:
  if test[cols].dtype == 'int64' or test[cols].dtype == 'float64':
    upper_range = test[cols].mean() + 3 * test[cols].std()
    lower_range = test[cols].mean() - 3 * test[cols].std()
    idx = test[(test[cols] > upper_range) | (test[cols] < lower_range)].index
    test = test.drop(idx)
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 988 entries, 0 to 999
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             988 non-null    int64  
 1   battery_power  988 non-null    int64  
 2   blue           988 non-null    int64  
 3   clock_speed    988 non-null    float64
 4   dual_sim       988 non-null    int64  
 5   fc             988 non-null    int64  
 6   four_g         988 non-null    int64  
 7   int_memory     988 non-null    int64  
 8   m_dep          988 non-null    float64
 9   mobile_wt      988 non-null    int64  
 10  n_cores        988 non-null    int64  
 11  pc             988 non-null    int64  
 12  px_height      988 non-null    int64  
 13  px_width       988 non-null    int64  
 14  ram            988 non-null    int64  
 15  sc_h           988 non-null    int64  
 16  sc_w           988 non-null    int64  
 17  talk_time      988 non-null    int64  
 18  three_g   

In [12]:
# delet columns without useful information for testdata
x_test = test.drop(columns = ['id'])
x_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 988 entries, 0 to 999
Data columns (total 20 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   battery_power  988 non-null    int64  
 1   blue           988 non-null    int64  
 2   clock_speed    988 non-null    float64
 3   dual_sim       988 non-null    int64  
 4   fc             988 non-null    int64  
 5   four_g         988 non-null    int64  
 6   int_memory     988 non-null    int64  
 7   m_dep          988 non-null    float64
 8   mobile_wt      988 non-null    int64  
 9   n_cores        988 non-null    int64  
 10  pc             988 non-null    int64  
 11  px_height      988 non-null    int64  
 12  px_width       988 non-null    int64  
 13  ram            988 non-null    int64  
 14  sc_h           988 non-null    int64  
 15  sc_w           988 non-null    int64  
 16  talk_time      988 non-null    int64  
 17  three_g        988 non-null    int64  
 18  touch_scre

In [13]:
# split future and target in train data
y = train['price_range']
x_train = train.drop(columns = ['price_range'])
print('x_train.shape =' , x_train.shape)
print('y.shape =' , y.shape)
x_train.info()

x_train.shape = (1988, 20)
y.shape = (1988,)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1988 entries, 0 to 1999
Data columns (total 20 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   battery_power  1988 non-null   float64
 1   blue           1988 non-null   float64
 2   clock_speed    1988 non-null   float64
 3   dual_sim       1988 non-null   float64
 4   fc             1988 non-null   float64
 5   four_g         1988 non-null   float64
 6   int_memory     1988 non-null   float64
 7   m_dep          1988 non-null   float64
 8   mobile_wt      1988 non-null   float64
 9   n_cores        1988 non-null   float64
 10  pc             1988 non-null   float64
 11  px_height      1988 non-null   float64
 12  px_width       1988 non-null   float64
 13  ram            1988 non-null   float64
 14  sc_h           1988 non-null   float64
 15  sc_w           1988 non-null   float64
 16  talk_time      1988 non-null   float64
 17  three_g

In [14]:
# Create a Radial basis function SVM classifier
svclassifier = svm.SVC(kernel='rbf')
svclassifier.fit(x_train, y)
y_train, y_test = tts( y , test_size = 0.4965, random_state = 0)
pred = svclassifier.predict(x_test)
print(confusion_matrix(y_test,pred))
print(classification_report(y_test,pred))

[[  0 252   0   0]
 [  0 247   0   0]
 [  0 245   0   0]
 [  0 244   0   0]]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       252
           1       0.25      1.00      0.40       247
           2       0.00      0.00      0.00       245
           3       0.00      0.00      0.00       244

    accuracy                           0.25       988
   macro avg       0.06      0.25      0.10       988
weighted avg       0.06      0.25      0.10       988



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [15]:
# Create a polynomial SVM classifier
svclassifier = svm.SVC(kernel='poly')
svclassifier.fit(x_train, y)
y_train, y_test = tts( y , test_size = 0.4965, random_state = 0)
pred = svclassifier.predict(x_test)
print(confusion_matrix(y_test,pred))
print(classification_report(y_test,pred))

[[  0   0   0 252]
 [  0   0   0 247]
 [  0   0   0 245]
 [  0   0   0 244]]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       252
           1       0.00      0.00      0.00       247
           2       0.00      0.00      0.00       245
           3       0.25      1.00      0.40       244

    accuracy                           0.25       988
   macro avg       0.06      0.25      0.10       988
weighted avg       0.06      0.25      0.10       988



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [16]:
# Create a linear SVM classifier
svclassifier = svm.SVC(kernel='linear')
svclassifier.fit(x_train, y)
y_train, y_test = tts( y , test_size = 0.4965, random_state = 0)
pred = svclassifier.predict(x_test)
print(confusion_matrix(y_test,pred))
print(classification_report(y_test,pred))

[[  0   0   0 252]
 [  0   0   0 247]
 [  0   0   0 245]
 [  0   0   0 244]]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       252
           1       0.00      0.00      0.00       247
           2       0.00      0.00      0.00       245
           3       0.25      1.00      0.40       244

    accuracy                           0.25       988
   macro avg       0.06      0.25      0.10       988
weighted avg       0.06      0.25      0.10       988



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [17]:
# Create a sigmoid SVM classifier
svclassifier = svm.SVC(kernel='sigmoid')
svclassifier.fit(x_train, y)
y_train, y_test = tts( y , test_size = 0.4965, random_state = 0)
pred = svclassifier.predict(x_test)
print(confusion_matrix(y_test,pred))
print(classification_report(y_test,pred))

[[  0   0  34 218]
 [  0   0  21 226]
 [  0   0  22 223]
 [  0   0  27 217]]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       252
           1       0.00      0.00      0.00       247
           2       0.21      0.09      0.13       245
           3       0.25      0.89      0.38       244

    accuracy                           0.24       988
   macro avg       0.11      0.24      0.13       988
weighted avg       0.11      0.24      0.13       988



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [18]:
# Create a linear SVM classifier with Hard Margin Classification
svclassifier = svm.SVC(kernel='linear' , C = 1000)
svclassifier.fit(x_train, y)
y_train, y_test = tts( y , test_size = 0.4965, random_state = 0)
pred = svclassifier.predict(x_test)
print(confusion_matrix(y_test,pred))
print(classification_report(y_test,pred))

[[  0   0   0 252]
 [  0   0   0 247]
 [  0   0   0 245]
 [  0   0   0 244]]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       252
           1       0.00      0.00      0.00       247
           2       0.00      0.00      0.00       245
           3       0.25      1.00      0.40       244

    accuracy                           0.25       988
   macro avg       0.06      0.25      0.10       988
weighted avg       0.06      0.25      0.10       988



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [19]:
# Create a linear SVM classifier with soft Margin Classification
svclassifier = svm.SVC(kernel='linear' , C = 0.1)
svclassifier.fit(x_train, y)
y_train, y_test = tts( y , test_size = 0.4965, random_state = 0)
pred = svclassifier.predict(x_test)
print(confusion_matrix(y_test,pred))
print(classification_report(y_test,pred))

[[  0   0   0 252]
 [  0   0   0 247]
 [  0   0   0 245]
 [  0   0   0 244]]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       252
           1       0.00      0.00      0.00       247
           2       0.00      0.00      0.00       245
           3       0.25      1.00      0.40       244

    accuracy                           0.25       988
   macro avg       0.06      0.25      0.10       988
weighted avg       0.06      0.25      0.10       988



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
